In [1]:
import shutil

import kuzu
import polars as pl
import watermark

import open_sanctions as os
import open_ownership as oo
import process_senzing as sz
# For visualizing the Kuzu graph in yFiles widget
from yfiles_jupyter_graphs_for_kuzu import KuzuGraphWidget

In [2]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2025-04-15T13:41:03.276364-04:00

Python implementation: CPython
Python version       : 3.13.2
IPython version      : 9.1.0

Compiler    : Clang 20.1.0 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 12
Architecture: 64bit

kuzu                          : 0.9.0
polars                        : 1.26.0
watermark                     : 2.5.0
yfiles_jupyter_graphs_for_kuzu: 0.0.3



In [3]:
DB_PATH = "./db"
shutil.rmtree(DB_PATH, ignore_errors=True)
db = kuzu.Database(DB_PATH)
conn = kuzu.Connection(db)

## Open Sanctions

In [4]:
df1 = pl.read_ndjson("data/open-sanctions.json")

In [5]:
df1.head(3)

DATA_SOURCE,RECORD_ID,RECORD_TYPE,LAST_CHANGE,NAMES,GENDER,RISKS,ADDRESSES,DATES,COUNTRIES,IDENTIFIERS,SOURCE_LINKS,RELATIONSHIPS,URL,CONTACTS
str,str,str,str,list[struct[3]],str,list[struct[1]],list[struct[7]],list[struct[2]],list[struct[3]],list[struct[9]],list[struct[1]],list[struct[5]],str,list[struct[1]]
"""OPEN_SANCTIONS""","""NK-25vyVFzt8vdJGgAXMRTwTJ""","""PERSON""","""2024-07-30T16:41:14""","[{""PRIMARY"",null,""Abassin BADSHAH""}]",null,"[{""corp.disqual""}]","[{""31 Quernmore Close, Bromley, Kent, United Kingdom, BR1 4EL"",null,null,null,null,null,null}]","[{null,""1985-05-12""}]","[{null,""gb"",null}]","[{null,null,null,null,""OPEN_SANCTIONS"",""NK-25vyVFzt8vdJGgAXMRTwTJ"",null,null,null}]","[{""https://find-and-update.company-information.service.gov.uk/disqualified-officers/natural/mGquuTbmESWiRmHJPz1ObUwfDgk""}]","[{null,null,""Directorship"",""OPEN_SANCTIONS"",""NK-SKAADAiqiZ78JsJjeg72Te""}, {null,null,""Directorship"",""OPEN_SANCTIONS"",""NK-3p3mmVWmjwVtTfKchz4kNE""}]","""https://www.opensanctions.org/…",null
"""OPEN_SANCTIONS""","""NK-3p3mmVWmjwVtTfKchz4kNE""","""ORGANIZATION""","""2025-01-07T00:33:03""","[{""PRIMARY"",""LMAR (GB) LTD"",null}]",null,null,"[{""31 Quernmore Close, Bromley, Kent, United Kingdom, BR1 4EL"",null,null,null,null,null,""BUSINESS""}]",null,"[{""gb"",null,null}]","[{null,null,null,null,""OPEN_SANCTIONS"",""NK-3p3mmVWmjwVtTfKchz4kNE"",null,null,null}]",null,"[{""OPEN_SANCTIONS"",""NK-3p3mmVWmjwVtTfKchz4kNE"",null,null,null}]","""https://www.opensanctions.org/…",null
"""OPEN-SANCTIONS""","""NK-auyPsLrBzRoxjCRWgjBvas""","""ORGANIZATION""","""2024-03-03T19:51:29""","[{""PRIMARY"",""WANDLE HOLDINGS LIMITED"",null}]",null,"[{""sanction.linked""}]","[{""DEANA BEACH APTS, BLOCK A, Flat 212, Προμαχών Ελευθερίας, 33, 'Αγιος Αθανάσιος, 4103, Λεμεσός, Κύπρος"",null,null,null,null,null,""BUSINESS""}]","[{""2006-12-08"",null}]","[{""cy"",null,null}]","[{null,null,""C188266"",null,null,null,null,null,null}, {null,null,""HE188266"",null,null,null,null,null,null}, {null,null,null,null,""OPEN-SANCTIONS"",""NK-auyPsLrBzRoxjCRWgjBvas"",null,null,null}]",null,"[{""OPEN-SANCTIONS"",""NK-auyPsLrBzRoxjCRWgjBvas"",null,null,null}]","""https://opensanctions.org/enti…",null


Each entity ID from Open Sanctions has a risk classification. This can be useful to associate an ID with a particular risk, allowing us to narrow down on candidates that are relevant to a particular investigation.

In [6]:
# Get risks from open sanctions
df_risk = os.extract_risks(df1)
df_risk.head(3)

id,topic
str,str
"""NK-25vyVFzt8vdJGgAXMRTwTJ""","""corp.disqual"""
"""NK-auyPsLrBzRoxjCRWgjBvas""","""sanction.linked"""
"""NK-cf4Q3KcmUnQbt8Cy7iTtwK""","""sanction.linked"""


We're now ready to extract the open sanctions data. The `extract_open_sanctions` function will take the raw data, process the nested fields within it and return the relevant columns that we need for our graph.

In [7]:
df_os = os.extract_open_sanctions(df1)
df_os.head(3)

id,kind,name,addr,url
str,str,str,str,str
"""NK-25vyVFzt8vdJGgAXMRTwTJ""","""PERSON""","""Abassin BADSHAH""","""31 Quernmore Close, Bromley, K…","""https://www.opensanctions.org/…"
"""NK-3p3mmVWmjwVtTfKchz4kNE""","""ORGANIZATION""","""LMAR (GB) LTD""","""31 Quernmore Close, Bromley, K…","""https://www.opensanctions.org/…"
"""NK-L2UmsZtsyvYiaEmHSaiZ2t""","""PERSON""","""Gulnara Suleimanova KERIMOVA""","""MOSCOW, RUS, 123430""","""https://www.opensanctions.org/…"


Of particular interest for this workshop is the person "Abassin Badshah", owner of multiple Papa John's franchises in London, who is known for his tax evasion conviction in 2021.

## Open Ownership

Open Ownership describes _ultimate beneficial ownership_ (UBO) details, which provides the "link" category of data.


In [8]:
df2 = pl.read_ndjson("data/open-ownership.json")
df2.head(3)

DATA_SOURCE,RECORD_ID,statementDate,RECORD_TYPE,NAMES,PRIMARY_NAME_FULL,personType,ATTRIBUTES,ADDRESSES,IDENTIFIERS,LINKS,RELATIONSHIPS,replaces_statements,REGISTRATION_DATE,dissolutionDate,REGISTRATION_COUNTRY,DATE_OF_BIRTH
str,str,str,str,list[struct[2]],str,str,list[struct[1]],list[struct[3]],list[struct[3]],list[struct[3]],list[struct[7]],list[struct[1]],str,str,str,str
"""OPEN-OWNERSHIP""","""10094521532396971848""","""2023-06-18""","""ORGANIZATION""","[{null,""GOLD WYNN UK HOLDINGS LIMITED""}]",null,null,null,"[{""BUSINESS"",""C/O Fladgate Llp, 16 Great Queen Street, London, WC2B 5DG"",""GB""}]","[{""12524623"",""GB-COH"",""GBR""}]","[{null,""https://opencorporates.com/companies/gb/12524623"",null}, {""https://register.openownership.org/entities/18432059995972240708"",null,null}]","[{""OOR"",""10094521532396971848"",null,null,null,null,null}, {null,null,""OOR"",""7584591804488095167"",""shareholding 75% 100%"",""2020-03-18"",""2020-04-29""}, … {null,null,""OOR"",""7584591804488095167"",""appointment_of_board"",""2020-03-18"",""2020-04-29""}]",null,"""2020-03-18""",null,"""GB""",null
"""OPEN-OWNERSHIP""","""10165632722354515453""","""2023-06-18""","""ORGANIZATION""","[{null,""UPSIDE TECHNOLOGY LIMITED""}]",null,null,null,"[{""BUSINESS"",""Apt 52, 3 Whitehall Court, London, SW1A 2EL"",""GB""}]","[{""12165794"",""GB-COH"",""GBR""}]","[{null,""https://opencorporates.com/companies/gb/12165794"",null}, {""https://register.openownership.org/entities/15659422647652524790"",null,null}]","[{""OOR"",""10165632722354515453"",null,null,null,null,null}, {null,null,""OOR"",""598161773989218568"",""shareholding 75% 100%"",""2019-08-20"",null}, … {null,null,""OOR"",""598161773989218568"",""appointment_of_board"",""2019-08-20"",null}]",null,"""2019-08-20""","""2022-10-11""","""GB""",null
"""OPEN-OWNERSHIP""","""10165632722354515453""","""2023-06-18""","""ORGANIZATION""","[{null,""UPSIDE TECHNOLOGY LIMITED""}]",null,null,null,"[{""BUSINESS"",""Apt 52, 3 Whitehall Court, London, SW1A 2EL"",""GB""}]","[{""12165794"",""GB-COH"",""GBR""}]","[{null,""https://opencorporates.com/companies/gb/12165794"",null}, {""https://register.openownership.org/entities/15659422647652524790"",null,null}]","[{""OOR"",""10165632722354515453"",null,null,null,null,null}, {null,null,""OOR"",""598161773989218568"",""shareholding 75% 100%"",""2019-08-20"",null}, … {null,null,""OOR"",""598161773989218568"",""appointment_of_board"",""2019-08-20"",null}]",null,"""2019-08-20""","""2022-10-11""","""GB""",null


Just like the Open Sanctions data, we can use the `extract_open_ownership` function to process the nested JSON data and return the relevant columns that we need for our graph.

In [9]:
df_oo = oo.extract_open_ownership(df2)
df_oo.head(3)

id,kind,name,address,country
str,str,str,str,str
"""10094521532396971848""","""ORGANIZATION""","""GOLD WYNN UK HOLDINGS LIMITED""","""C/O Fladgate Llp, 16 Great Que…","""GB"""
"""10165632722354515453""","""ORGANIZATION""","""UPSIDE TECHNOLOGY LIMITED""","""Apt 52, 3 Whitehall Court, Lon…","""GB"""
"""10264459789712927869""","""PERSON""","""Kenneth Kurt Hansen""","""Finderupvej 61, Kastrup, 2770""","""DK"""


For the relationships in our graph, we'll need to select only the relationships that have **both** `src` and `dst` in the list of ids. This is done via the `extract_open_ownership_relationships` function.

In [10]:
ids = df_oo.select("id").to_series().to_list()
df_oa_relationships = oo.extract_open_ownership_relationships(df2, open_ownership_ids=ids)
df_oa_relationships.head(3)

src_id,dst_id,role,date
str,str,str,str
"""10094521532396971848""","""7584591804488095167""","""shareholding 75% 100%""","""2020-03-18"""
"""10094521532396971848""","""7584591804488095167""","""appointment_of_board""","""2020-03-18"""
"""10094521532396971848""","""7584591804488095167""","""voting_rights 75% 100%""","""2020-03-18"""


## Senzing workflow

To generate high quality entity data, we'll use Senzing to process the Open Sanctions and Open Ownership data via the Senzing Python SDK. The returned data from Senzing contains resolved entities (in `export.json`), which is once again nested JSON that we can use to create our graph.

In [11]:
df3 = pl.read_ndjson("data/export.json")
df3.head(3)

RESOLVED_ENTITY,RELATED_ENTITIES
struct[2],list[struct[8]]
"{1,[{""OPEN-SANCTIONS"",""NK-25vyVFzt8vdJGgAXMRTwTJ"",""GENERIC"",1,""49B1F5D5BB70A51ECD1E5DED69A3F99F46232F8D"",""Abassin BADSHAH"","""",0,"""","""",""2025-02-02 22:33:52.027""}, {""OPEN-OWNERSHIP"",""17207853441353212969"",""GENERIC"",100121,""4237F29B220116BE634753371BE3529B08F97201"",""Abassin Badshah"",""+NAME+ADDRESS+NATIONALITY"",1,""RESOLVED"",""CNAME_CFF"",""2025-02-02 22:34:26.260""}, {""OPEN-OWNERSHIP"",""6747548100436839873"",""GENERIC"",100211,""793D2A980A44DD047ECD8603B2CC8E2045F59858"",""Abassin Badshah"",""+NAME+DOB+NATIONALITY"",1,""RESOLVED"",""SNAME_SSTAB"",""2025-02-02 22:34:26.994""}]}","[{2,11,""DISCLOSED"",""+OPEN-SANCTIONS(DIRECTORSHIP:)"",""DISCLOSED"",1,0,[{""OPEN-SANCTIONS"",""NK-3p3mmVWmjwVtTfKchz4kNE""}]}, {9,11,""DISCLOSED"",""+ADDRESS+OOR(:SHAREHOLDING 50% 75%,VOTING_RIGHTS 50% 75%)+OPEN-SANCTIONS(DIRECTORSHIP:)-RECORD_TYPE"",""DISCLOSED"",3,0,[{""OPEN-SANCTIONS"",""NK-SKAADAiqiZ78JsJjeg72Te""}, {""OPEN-OWNERSHIP"",""17892212632775799925""}]}, … {100132,3,""POSSIBLY_RELATED"",""+ADDRESS+NATIONALITY"",""CFF"",0,0,[{""OPEN-OWNERSHIP"",""17952626459015773513""}]}]"
"{2,[{""OPEN-SANCTIONS"",""NK-3p3mmVWmjwVtTfKchz4kNE"",""GENERIC"",2,""5D0549B7E9CB8BBE38DF3B595650C56C4A35BB7C"",""LMAR (GB) LTD"","""",0,"""","""",""2025-02-02 22:33:52.034""}]}","[{1,11,""DISCLOSED"",""+OPEN-SANCTIONS(:DIRECTORSHIP)"",""DISCLOSED"",1,0,[{""OPEN-SANCTIONS"",""NK-25vyVFzt8vdJGgAXMRTwTJ""}, {""OPEN-OWNERSHIP"",""17207853441353212969""}, {""OPEN-OWNERSHIP"",""6747548100436839873""}]}]"
"{3,[{""OPEN-SANCTIONS"",""NK-auyPsLrBzRoxjCRWgjBvas"",""GENERIC"",3,""CE753D92E4AE9DA3A712F4846CF374D6243098E2"",""WANDLE HOLDINGS LIMITED"","""",0,"""","""",""2025-02-02 22:33:52.037""}]}","[{12,11,""DISCLOSED"",""+OPEN-SANCTIONS(:BENEFICIARY)"",""DISCLOSED"",1,0,[{""OPEN-SANCTIONS"",""Q44155839""}]}]"


Due to the deeply nested nature of the data, we'll have to process the data in a few steps using the `process_senzing_export` function. Each entity is assigned a unique identifier with the prefix `sz_`, associated with the record ID from the original data, and which data source it comes from.

In [12]:
df_sz = sz.process_senzing_export(df3)
df_sz.head(3)

ent_id,rec_id,source,descrip,level,why
str,str,str,str,i8,str
"""sz_1""","""NK-25vyVFzt8vdJGgAXMRTwTJ""","""OPEN-SANCTIONS""","""Abassin BADSHAH""",0,null
"""sz_1""","""17207853441353212969""","""OPEN-OWNERSHIP""","""Abassin Badshah""",1,"""+NAME+ADDRESS+NATIONALITY"""
"""sz_1""","""6747548100436839873""","""OPEN-OWNERSHIP""","""Abassin Badshah""",1,"""+NAME+DOB+NATIONALITY"""


We'll need to create `Matched` relationships between the entities in our graph. This is done via the `extract_related_entities` function. The from/to columns in the following table are the source and destination Senzing identifiers.

In [13]:
df_rel = sz.extract_related_entities(df3)
df_rel.head(3)


ent_id,rel_id,why,level
str,str,str,i8
"""sz_1""","""sz_2""","""+OPEN-SANCTIONS(DIRECTORSHIP:)""",11
"""sz_1""","""sz_9""","""+ADDRESS+OOR(:SHAREHOLDING 50%…",11
"""sz_1""","""sz_100075""","""+ADDRESS+OOR(:APPOINTMENT_OF_B…",11


The entity DataFrame `df_sz` contains multiple records for each entity, so we need to ensure that we only include one record per entity.

In [14]:
df_ent = (
    # df_sz.filter(pl.col("why").is_not_null())  # Throw away entities that don't have a "why"
    df_sz.unique(subset=["ent_id"])
    .select("ent_id", "descrip")
    .sort("ent_id")
)

df_ent.head(3)

ent_id,descrip
str,str
"""sz_1""","""Abassin BADSHAH"""
"""sz_10""","""ニコラス・トマス・ライト"""
"""sz_100001""","""GOLD WYNN UK HOLDINGS LIMITED"""


### Separate Senzing entities by source

The final step to preprocess the data for our graph is to separate the entities by their source (whether they are from Open Sanctions or Open Ownership).

In [15]:
df_sz_os = df_sz.filter(pl.col("source") == "OPEN-SANCTIONS").select("ent_id", "rec_id", "why", "level")
df_sz_oo = df_sz.filter(pl.col("source") == "OPEN-OWNERSHIP").select("ent_id", "rec_id", "why", "level")

## Copy data to Kuzu graph

Kuzu is an embedded, open source graph database that supports the Cypher query language. It uses a structured property graph model, which is similar to the labelled property graph model you may be familiar with from other systems -- the only difference being that Kuzu requires strict data types for properties in the schema.

The following steps will create the graph schema in Kuzu (node and relationship tables) and copy the data into them.

In [16]:
# Create a yFiles graph widget so we can explore our graph as it's created
g = KuzuGraphWidget(conn)

In [17]:
conn.execute("CREATE NODE TABLE IF NOT EXISTS OpenSanctions (id STRING PRIMARY KEY, kind STRING, name STRING, addr STRING, url STRING)")
conn.execute("CREATE NODE TABLE IF NOT EXISTS OpenOwnership (id STRING PRIMARY KEY, kind STRING, name STRING, addr STRING, country STRING)")
conn.execute("CREATE NODE TABLE IF NOT EXISTS Risk (topic STRING PRIMARY KEY)")
conn.execute("CREATE NODE TABLE IF NOT EXISTS Entity (id STRING PRIMARY KEY, descrip STRING)")
conn.execute("CREATE REL TABLE IF NOT EXISTS Role (FROM OpenOwnership TO OpenOwnership, role STRING, date DATE)")

In [18]:
conn.execute("COPY OpenSanctions FROM df_os")
conn.execute("COPY OpenOwnership FROM df_oo")
conn.execute("COPY Risk FROM (LOAD FROM df_risk RETURN DISTINCT topic)")
conn.execute("COPY Entity FROM (LOAD FROM df_ent RETURN ent_id AS id, descrip)")
conn.execute("COPY Role FROM df_oa_relationships")


In [19]:
g.show_cypher("MATCH (a:OpenSanctions:OpenOwnership)-[b]->(c:OpenSanctions:OpenOwnership) RETURN * LIMIT 50")

GraphWidget(layout=Layout(height='800px', width='100%'))

In [20]:
# Create Related table between entities
conn.execute("CREATE REL TABLE IF NOT EXISTS Related (FROM Entity TO Entity, why STRING, level INT8)");
conn.execute("COPY Related FROM df_rel");

In [21]:
g.show_cypher(
    """
    MATCH (a:Entity)-[b *1..3]->(c)
    WHERE a.descrip CONTAINS "Abassin"
    RETURN * LIMIT 50
    """
)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [22]:
# Create Matched table between multiple sets of entities
conn.execute(
    """
    CREATE REL TABLE IF NOT EXISTS Matched (
        FROM Entity TO OpenSanctions,
        FROM Entity TO OpenOwnership,
        why STRING,
        level INT8
    )
"""
)
conn.execute("COPY Matched FROM df_sz_os (from='Entity', to='OpenSanctions')");
conn.execute("COPY Matched FROM df_sz_oo (from='Entity', to='OpenOwnership')");

In [23]:
# Add Risks to OpenSanctions
conn.execute("CREATE REL TABLE IF NOT EXISTS HasRisk (FROM OpenSanctions TO Risk)")
conn.execute("COPY HasRisk FROM df_risk")

In [24]:
g.show_cypher(
    """
    MATCH (a:Entity)-[b *1..3]->(c)
    WHERE a.descrip CONTAINS "Abassin"
    RETURN * LIMIT 50
    """
)

GraphWidget(layout=Layout(height='630px', width='100%'))

We've successfully combined the data from Open Sanctions, Open Ownership, and resolved entities from Senzing to create a graph that's persisted in Kuzu!
This graph is of high enough quality that it can be used for a variety of investigative tasks downstream. Happy querying!